In [1]:
import numpy as np
import pandas as pd

In [12]:
file = 'global_data_10_0.5'
file_split = np.array(file.split('_'))[-2:].astype(float)

time_bin  = file_split[-1]
H_bin  = file_split[-2]

time_bins = np.arange(0,24,time_bin)
H_bins = np.arange(90,130,H_bin)

shape = len(time_bins) * len(H_bins)

file_path = f'../../Output/{file}.csv'
supermag_path = '../../Index/supermag_global.csv'

In [13]:
data_df = pd.read_csv(file_path)
supermag_df = pd.read_csv(supermag_path)
Date = np.array(data_df['Date']).reshape(-1,shape)[:,0]
Time = np.array(data_df['Hours']).reshape(-1,shape)[:,0]
Mask = np.array(data_df['EXP Density']).reshape(-1,shape) == 0.0

Date_index = supermag_df['Date'].values
Time_index = supermag_df['Time'].values
SML = supermag_df['SML'].values
SMU = supermag_df['SMU'].values
SME = SMU - SML

if time_bin == 1:
    Date_index = Date_index.reshape(-1,2)[:,0]
    Time_index = Time_index.reshape(-1,2)[:,0]
    SML = SML.reshape(-1,2)[:,0]
    SMU = SMU.reshape(-1,2)[:,0]
    SME = SME.reshape(-1,2)[:,0]

next_SML = np.roll(SML, -1)
next_SMU = np.roll(SMU, -1)
next_SME = np.roll(SME, -1)

dSML = next_SML-SML
dSMU = next_SMU-SMU
dSME = next_SME-SME

SML_list = []
dSML_list = []
SMU_list = []
dSMU_list = []
SME_list = []
dSME_list = []

for date in Date:
    date_index_mask = Date_index == date

    SML_list.append(np.tile(SML[date_index_mask], int(40/H_bin)))
    dSML_list.append(np.tile(dSML[date_index_mask], int(40/H_bin)))
    SMU_list.append(np.tile(SMU[date_index_mask], int(40/H_bin)))
    dSMU_list.append(np.tile(dSMU[date_index_mask], int(40/H_bin)))
    SME_list.append(np.tile(SME[date_index_mask], int(40/H_bin)))
    dSME_list.append(np.tile(dSME[date_index_mask], int(40/H_bin)))


SML_array = np.array(SML_list)
dSML_array = np.array(dSML_list)
SMU_array = np.array(SMU_list)
dSMU_array = np.array(dSMU_list)
SME_array = np.array(SME_list)
dSME_array = np.array(dSME_list)

SML_array[Mask] = 0
dSML_array[Mask] = 0
SMU_array[Mask] = 0
dSMU_array[Mask] = 0
SME_array[Mask] = 0
dSME_array[Mask] = 0

SML_array = SML_array.reshape(1, -1)[0]
dSML_array = dSML_array.reshape(1, -1)[0]
SMU_array = SMU_array.reshape(1, -1)[0]
dSMU_array = dSMU_array.reshape(1, -1)[0]
SME_array = SME_array.reshape(1, -1)[0]
dSME_array = dSME_array.reshape(1, -1)[0]

data_df['SML'] = (SML_array*100).astype(int)/100
data_df['SML Gradient'] = (dSML_array*100).astype(int)/100
data_df['SMU'] = (SMU_array*100).astype(int)/100
data_df['SMU Gradient'] = (dSMU_array*100).astype(int)/100
data_df['SME'] = (SME_array*100).astype(int)/100
data_df['SME Gradient'] = (dSME_array*100).astype(int)/100

data_df.to_csv(file_path, index=False)